# SQUEEZENET

In [1]:
from tqdm import tqdm
import numpy as np
import csv
import os
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
from keras.models import Model
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from random import uniform
import bcolz
from time import *
from keras.layers import merge
from keras.layers.core import Lambda
from keras.models import Model
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras.layers import Input, Dense
import math
import pandas as pd
from keras.utils import np_utils as u
import utils; reload(utils)
from utils import *
import resnet50; reload(resnet50)
from resnet50 import Resnet50
from keras.preprocessing.image import ImageDataGenerator
import myFunctionsv3; reload(myFunctionsv3)
from myFunctionsv3 import *
from  keras.applications.resnet50 import ResNet50
#from squeezenet import SqueezeNet

import gc

%matplotlib inline


/home/ubuntu/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: unknown error)


# Training

In [2]:
nf=128; p=0.01
def get_lrg_layers():
    
    #inp = Input((21, 39, 512))
    inp = Input((31, 56, 512))
    x = BatchNormalization(axis=1)(inp)
    x = MaxPooling2D()(x)
    x = Conv2D(nf,(1,1), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = MaxPooling2D()(x)
    x = Conv2D(nf,(1,1), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = MaxPooling2D()(x)
    x = Conv2D(nf,(1,1), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    
    x_15 = Conv2D(15,(3,3), activation='relu', padding='same')(x)
    x_15 = BatchNormalization(axis=1)(x_15)
    x_15 = GlobalMaxPooling2D()(x_15)
    x_sig_15 = Activation('sigmoid')(x_15)
    
    x_4 = Conv2D(4,(3,3), activation='relu', padding='same')(x)
    x_4 = BatchNormalization(axis=1)(x_4)
    x_4 = GlobalMaxPooling2D()(x_4)
    x_sof_4 = Activation('sigmoid')(x_4)
    
    x_2 = Conv2D(2,(3,3), activation='relu', padding='same')(x)
    x_2 = BatchNormalization(axis=1)(x_2)
    x_2 = GlobalMaxPooling2D()(x_2)
    x_sof_2 = Activation('sigmoid')(x_2)
    
    model = Model([inp], [x_sig_15, x_sof_4, x_sof_2])
    
    return model

In [3]:
lrg_model = get_lrg_layers()

In [4]:
lrg_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 31, 56, 512)   0                                            
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 31, 56, 512)   124                                          
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 15, 28, 512)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 15, 28, 128)   65664                                        
___________________________________________________________________________________________

In [5]:
gc.collect()

24724

In [15]:
nf=128; p=0.01
def get_lrg_layers_2():
    
    inp = Input((31, 56, 512))
    x = BatchNormalization(axis=1)(inp)
    x = MaxPooling2D()(x)
    x = Conv2D(nf,(1,1), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = MaxPooling2D()(x)
    x = Conv2D(nf,(1,1), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = MaxPooling2D()(x)
    x = Conv2D(nf,(1,1), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)
    
    x_9 = Conv2D(9,(3,3), activation='relu', padding='same')(x)
    x_9 = BatchNormalization(axis=1)(x_9)
    x_9 = GlobalMaxPooling2D()(x_9)
    x_sig_9 = Activation('sigmoid')(x_9)
    
    x_6 = Conv2D(6,(3,3), activation='relu', padding='same')(x)
    x_6 = BatchNormalization(axis=1)(x_6)
    x_6 = GlobalMaxPooling2D()(x_6)
    x_sig_6 = Activation('sigmoid')(x_6)
    
    x_4 = Conv2D(5,(3,3), activation='relu', padding='same')(x)
    x_4 = BatchNormalization(axis=1)(x_4)
    x_4 = GlobalMaxPooling2D()(x_4)
    x_sof_4 = Activation('softmax')(x_4)
    
    x_2 = Conv2D(3,(3,3), activation='relu', padding='same')(x)
    x_2 = BatchNormalization(axis=1)(x_2)
    x_2 = GlobalMaxPooling2D()(x_2)
    x_sof_2 = Activation('softmax')(x_2)
    
    model = Model([inp], [x_sig_9,x_sig_6, x_sof_4, x_sof_2])
    
    return model

In [16]:
lrg_model2 = get_lrg_layers_2()

In [17]:
lrg_model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 31, 56, 512)  0                                            
__________________________________________________________________________________________________
batch_normalization_38 (BatchNo (None, 31, 56, 512)  124         input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 15, 28, 512)  0           batch_normalization_38[0][0]     
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 15, 28, 128)  65664       max_pooling2d_10[0][0]           
__________________________________________________________________________________________________
batch_norm

# Model I7

In [9]:
lrg_model2.load_weights('/cat/home/ubuntu/cat/out/weights/sqznt_weights_folds_8_9_128_Iteration8.hdf5')

6,10,11,12,13,14,15,16,17,0,5,9,18,19,20,1,2,3,4,7,8

In [19]:
files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19_val_minus_0p5','train24_val_minus_0p5']
FIRST_TIME=True
for i in range(len(files)):
    conv_val_feat = load_array('/cat/home/ubuntu/cat/out/features/train/'+files[i]+'_sqznt.dat')
    labels_val = load_array('/cat/home/ubuntu/cat/in/train/labels/'+files[i]+'_labels.dat')
    if FIRST_TIME:
        predictions_all = lrg_model2.predict(conv_val_feat) 
        predictions_all = np.hstack((predictions_all[0][:,0:9],predictions_all[1][:,0:6],predictions_all[2][:,0:4],predictions_all[3][:,0:2]))
        labels_all = labels_val
        FIRST_TIME = False
    else:
        temp = lrg_model2.predict(conv_val_feat)
        temp = np.hstack((temp[0][:,0:9],temp[1][:,0:6],temp[2][:,0:4],temp[3][:,0:2]))
        predictions_all = np.vstack((predictions_all,temp))
        labels_all = np.vstack((labels_all,labels_val))
    del(conv_val_feat,labels_val)
    gc.collect()
print predictions_all.shape, labels_all.shape
predictions_center = np.copy(predictions_all)

(28900, 21) (28900, 23)


In [20]:
label_files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19_val_minus_0p5','train24_val_minus_0p5']

files = ['train03top_left_val_minus_0p5','train04top_left_val_minus_0p5','train10top_left_val_minus_0p5','train18top_left_val_minus_0p5',
         'train21top_left_val_minus_0p5','train14top_left_val_minus_0p5','train19top_left_val_minus_0p5','train24top_left_val_minus_0p5']
FIRST_TIME=True
for i in range(len(files)):
    conv_val_feat = load_array('/cat/home/ubuntu/cat/out/features/train/'+files[i]+'_sqznt.dat')
    labels_val = load_array('/cat/home/ubuntu/cat/in/train/labels/'+label_files[i]+'_labels.dat')
    if FIRST_TIME:
        predictions_all = lrg_model2.predict(conv_val_feat) 
        predictions_all = np.hstack((predictions_all[0][:,0:9],predictions_all[1][:,0:6],predictions_all[2][:,0:4],predictions_all[3][:,0:2]))
        labels_all = labels_val
        FIRST_TIME = False
    else:
        temp = lrg_model2.predict(conv_val_feat)
        temp = np.hstack((temp[0][:,0:9],temp[1][:,0:6],temp[2][:,0:4],temp[3][:,0:2]))
        predictions_all = np.vstack((predictions_all,temp))
        labels_all = np.vstack((labels_all,labels_val))
    del(conv_val_feat,labels_val)
    gc.collect()
print predictions_all.shape, labels_all.shape
predictions_top_left = np.copy(predictions_all)

(28900, 21) (28900, 23)


In [21]:
label_files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19_val_minus_0p5','train24_val_minus_0p5']
files = ['train03top_right_val_minus_0p5','train04top_right_val_minus_0p5','train10top_right_val_minus_0p5','train18top_right_val_minus_0p5',
         'train21top_right_val_minus_0p5','train14top_right_val_minus_0p5','train19top_right_val_minus_0p5','train24top_right_val_minus_0p5']
FIRST_TIME=True
for i in range(len(files)):
    conv_val_feat = load_array('/cat/home/ubuntu/cat/out/features/train/'+files[i]+'_sqznt.dat')
    labels_val = load_array('/cat/home/ubuntu/cat/in/train/labels/'+label_files[i]+'_labels.dat')
    if FIRST_TIME:
        predictions_all = lrg_model2.predict(conv_val_feat) 
        predictions_all = np.hstack((predictions_all[0][:,0:9],predictions_all[1][:,0:6],predictions_all[2][:,0:4],predictions_all[3][:,0:2]))
        labels_all = labels_val
        FIRST_TIME = False
    else:
        temp = lrg_model2.predict(conv_val_feat)
        temp = np.hstack((temp[0][:,0:9],temp[1][:,0:6],temp[2][:,0:4],temp[3][:,0:2]))
        predictions_all = np.vstack((predictions_all,temp))
        labels_all = np.vstack((labels_all,labels_val))
    del(conv_val_feat,labels_val)
    gc.collect()
print predictions_all.shape, labels_all.shape
predictions_top_right = np.copy(predictions_all)

(28900, 21) (28900, 23)


In [22]:
label_files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19_val_minus_0p5','train24_val_minus_0p5']
files = ['train03bottom_right_val_minus_0p5','train04bottom_right_val_minus_0p5','train10bottom_right_val_minus_0p5','train18bottom_right_val_minus_0p5',
         'train21bottom_right_val_minus_0p5','train14bottom_right_val_minus_0p5','train19bottom_right_val_minus_0p5','train24bottom_right_val_minus_0p5']
FIRST_TIME=True
for i in range(len(files)):
    conv_val_feat = load_array('/cat/home/ubuntu/cat/out/features/train/'+files[i]+'_sqznt.dat')
    labels_val = load_array('/cat/home/ubuntu/cat/in/train/labels/'+label_files[i]+'_labels.dat')
    if FIRST_TIME:
        predictions_all = lrg_model2.predict(conv_val_feat) 
        predictions_all = np.hstack((predictions_all[0][:,0:9],predictions_all[1][:,0:6],predictions_all[2][:,0:4],predictions_all[3][:,0:2]))
        labels_all = labels_val
        FIRST_TIME = False
    else:
        temp = lrg_model2.predict(conv_val_feat)
        temp = np.hstack((temp[0][:,0:9],temp[1][:,0:6],temp[2][:,0:4],temp[3][:,0:2]))
        predictions_all = np.vstack((predictions_all,temp))
        labels_all = np.vstack((labels_all,labels_val))
    del(conv_val_feat,labels_val)
    gc.collect()
print predictions_all.shape, labels_all.shape
predictions_bottom_right = np.copy(predictions_all)

(28900, 21) (28900, 23)


In [23]:
label_files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19_val_minus_0p5','train24_val_minus_0p5']
files = ['train03bottom_left_val_minus_0p5','train04bottom_left_val_minus_0p5','train10bottom_left_val_minus_0p5','train18bottom_left_val_minus_0p5',
         'train21bottom_left_val_minus_0p5','train14bottom_left_val_minus_0p5','train19bottom_left_val_minus_0p5','train24bottom_left_val_minus_0p5']
FIRST_TIME=True
for i in range(len(files)):
    conv_val_feat = load_array('/cat/home/ubuntu/cat/out/features/train/'+files[i]+'_sqznt.dat')
    labels_val = load_array('/cat/home/ubuntu/cat/in/train/labels/'+label_files[i]+'_labels.dat')
    if FIRST_TIME:
        predictions_all = lrg_model2.predict(conv_val_feat) 
        predictions_all = np.hstack((predictions_all[0][:,0:9],predictions_all[1][:,0:6],predictions_all[2][:,0:4],predictions_all[3][:,0:2]))
        labels_all = labels_val
        FIRST_TIME = False
    else:
        temp = lrg_model2.predict(conv_val_feat)
        temp = np.hstack((temp[0][:,0:9],temp[1][:,0:6],temp[2][:,0:4],temp[3][:,0:2]))
        predictions_all = np.vstack((predictions_all,temp))
        labels_all = np.vstack((labels_all,labels_val))
    del(conv_val_feat,labels_val)
    gc.collect()
print predictions_all.shape, labels_all.shape
predictions_bottom_left = np.copy(predictions_all)

(28900, 21) (28900, 23)


6,10,11,12,13,14,15,16,17,0,5,9,18,19,20,1,2,3,4,7,8

In [24]:
labels_all = labels_all[:,2:23]
labels_all[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]] = labels_all[:,[6,10,11,12,13,14,15,16,17,0,5,9,18,19,20,1,2,3,4,7,8]]

In [25]:
predictionals_all_patches = np.zeros([5, 21,predictions_center.shape[0]],dtype=np.float32)
for i in range(21):
    predictionals_all_patches[0,i] = predictions_center[:,i]
    predictionals_all_patches[1,i] = predictions_top_left[:,i]
    predictionals_all_patches[2,i] = predictions_top_right[:,i]
    predictionals_all_patches[3,i] = predictions_bottom_right[:,i]
    predictionals_all_patches[4,i] = predictions_bottom_left[:,i]
print predictionals_all_patches.shape

(5, 21, 28900)


In [26]:
predictionals_all_patches_average2 = np.zeros([1, 21,predictions_center.shape[0]],dtype=np.float32)
for i in range(21):
    predictionals_all_patches_average2[0,i] = (predictionals_all_patches[0,i] + predictionals_all_patches[1,i] + predictionals_all_patches[2,i] + predictionals_all_patches[3,i] + predictionals_all_patches[4,i])/5

In [29]:
predictionals_all_patches_average2 = predictions_center

In [27]:
predictionals_all_patches_average2 = predictionals_all_patches_average2[0]
predictionals_all_patches_average2 = np.transpose(predictionals_all_patches_average2,(1,0))

In [30]:
# Computing the area under the ROC curve
from sklearn.metrics import roc_curve, auc
from math import isnan

totalScore=0
count = 0

predictions_temp = np.copy(predictionals_all_patches_average2)
truth_temp = np.copy(labels_all)

for j in range(21):   
    index = 0
    #remove rows with label = 0.5
    for i in range(labels_all.shape[0]):
        if labels_all[i,j] != 0.5:
            truth_temp[index,j]=labels_all[i,j]
            predictions_temp[index,j]=predictionals_all_patches_average2[i,j]
            index += 1
    fpr, tpr, _ = roc_curve(truth_temp[0:index,j], predictions_temp[0:index,j])
    score = auc(fpr, tpr)
    print j, score
    if isnan(score):
        score=0.0
    else:
        totalScore+=score
        count +=1
    
print totalScore/count

0 0.5
1 0.998922162599
2 0.926481170686
3 0.968421606379
4 0.5
5 0.5
6 0.858570697198
7 0.936598582618
8 0.931129496064
9 0.5
10 0.260520227289
11 0.63500805487
12 0.602991919822
13 1.0
14 0.374429634759
15 nan
16 0.776779227354
17 0.828757151304
18 0.878686621061
19 0.954246891119
20 0.938328171593
0.743493580736


# Iteration 10

In [6]:
lrg_model.load_weights('/cat/home/ubuntu/cat/out/weights/sqznt_weights_folds_8_9_128_Iteration13L.hdf5')

0,5,6,9,10,11,12,13,14,15,16,17,18,19,20,1,2,3,4,7,8

In [7]:
files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19A_val_minus_0p5','train19B_val_minus_0p5','train24_val_minus_0p5']
FIRST_TIME=True
for i in range(len(files)):
    conv_val_feat = load_array('/cat4/home/ubuntu/cat4/out/features/train/'+files[i]+'_sqznt.dat')
    labels_val = load_array('/cat/home/ubuntu/cat/in/train/labels/'+files[i]+'_labels.dat')
    if FIRST_TIME:
        predictions_all = lrg_model.predict(conv_val_feat) 
        predictions_all = np.hstack((predictions_all[0][:,0:15],predictions_all[1][:,0:4],predictions_all[2][:,0:2]))
        labels_all = labels_val
        FIRST_TIME = False
    else:
        temp = lrg_model.predict(conv_val_feat)
        temp = np.hstack((temp[0][:,0:15],temp[1][:,0:4],temp[2][:,0:2]))
        predictions_all = np.vstack((predictions_all,temp))
        labels_all = np.vstack((labels_all,labels_val))
    del(conv_val_feat,labels_val)
    gc.collect()
print predictions_all.shape, labels_all.shape
predictions_center = np.copy(predictions_all)

(31687, 21) (31687, 23)


In [8]:
label_files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19A_val_minus_0p5','train19B_val_minus_0p5','train24_val_minus_0p5']

files = ['train03top_left_val_minus_0p5','train04top_left_val_minus_0p5','train10top_left_val_minus_0p5','train18top_left_val_minus_0p5',
         'train21top_left_val_minus_0p5','train14top_left_val_minus_0p5','train19Atop_left_val_minus_0p5','train19Btop_left_val_minus_0p5','train24top_left_val_minus_0p5']

FIRST_TIME=True
for i in range(len(files)):
    conv_val_feat = load_array('/cat4/home/ubuntu/cat4/out/features/train/'+files[i]+'_sqznt.dat')
    labels_val = load_array('/cat/home/ubuntu/cat/in/train/labels/'+label_files[i]+'_labels.dat')
    if FIRST_TIME:
        predictions_all = lrg_model.predict(conv_val_feat) 
        predictions_all = np.hstack((predictions_all[0][:,0:15],predictions_all[1][:,0:4],predictions_all[2][:,0:2]))
        labels_all = labels_val
        FIRST_TIME = False
    else:
        temp = lrg_model.predict(conv_val_feat)
        temp = np.hstack((temp[0][:,0:15],temp[1][:,0:4],temp[2][:,0:2]))
        predictions_all = np.vstack((predictions_all,temp))
        labels_all = np.vstack((labels_all,labels_val))
    del(conv_val_feat,labels_val)
    gc.collect()
print predictions_all.shape, labels_all.shape
predictions_top_left = np.copy(predictions_all)

(31687, 21) (31687, 23)


In [9]:
label_files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19A_val_minus_0p5','train19B_val_minus_0p5','train24_val_minus_0p5']

files = ['train03top_right_val_minus_0p5','train04top_right_val_minus_0p5','train10top_right_val_minus_0p5','train18top_right_val_minus_0p5',
         'train21top_right_val_minus_0p5','train14top_right_val_minus_0p5','train19Atop_right_val_minus_0p5','train19Btop_right_val_minus_0p5','train24top_right_val_minus_0p5']

FIRST_TIME=True
for i in range(len(files)):
    conv_val_feat = load_array('/cat4/home/ubuntu/cat4/out/features/train/'+files[i]+'_sqznt.dat')
    labels_val = load_array('/cat/home/ubuntu/cat/in/train/labels/'+label_files[i]+'_labels.dat')
    if FIRST_TIME:
        predictions_all = lrg_model.predict(conv_val_feat) 
        predictions_all = np.hstack((predictions_all[0][:,0:15],predictions_all[1][:,0:4],predictions_all[2][:,0:2]))
        labels_all = labels_val
        FIRST_TIME = False
    else:
        temp = lrg_model.predict(conv_val_feat)
        temp = np.hstack((temp[0][:,0:15],temp[1][:,0:4],temp[2][:,0:2]))
        predictions_all = np.vstack((predictions_all,temp))
        labels_all = np.vstack((labels_all,labels_val))
    del(conv_val_feat,labels_val)
    gc.collect()
print predictions_all.shape, labels_all.shape
predictions_top_right = np.copy(predictions_all)

(31687, 21) (31687, 23)


In [10]:
label_files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19A_val_minus_0p5','train19B_val_minus_0p5','train24_val_minus_0p5']

files = ['train03bottom_right_val_minus_0p5','train04bottom_right_val_minus_0p5','train10bottom_right_val_minus_0p5','train18bottom_right_val_minus_0p5',
         'train21bottom_right_val_minus_0p5','train14bottom_right_val_minus_0p5','train19Abottom_right_val_minus_0p5','train19Bbottom_right_val_minus_0p5','train24bottom_right_val_minus_0p5']

FIRST_TIME=True
for i in range(len(files)):
    conv_val_feat = load_array('/cat4/home/ubuntu/cat4/out/features/train/'+files[i]+'_sqznt.dat')
    labels_val = load_array('/cat/home/ubuntu/cat/in/train/labels/'+label_files[i]+'_labels.dat')
    if FIRST_TIME:
        predictions_all = lrg_model.predict(conv_val_feat) 
        predictions_all = np.hstack((predictions_all[0][:,0:15],predictions_all[1][:,0:4],predictions_all[2][:,0:2]))
        labels_all = labels_val
        FIRST_TIME = False
    else:
        temp = lrg_model.predict(conv_val_feat)
        temp = np.hstack((temp[0][:,0:15],temp[1][:,0:4],temp[2][:,0:2]))
        predictions_all = np.vstack((predictions_all,temp))
        labels_all = np.vstack((labels_all,labels_val))
    del(conv_val_feat,labels_val)
    gc.collect()
print predictions_all.shape, labels_all.shape
predictions_bottom_right = np.copy(predictions_all)

(31687, 21) (31687, 23)


In [11]:
label_files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19A_val_minus_0p5','train19B_val_minus_0p5','train24_val_minus_0p5']

files = ['train03bottom_left_val_minus_0p5','train04bottom_left_val_minus_0p5','train10bottom_left_val_minus_0p5','train18bottom_left_val_minus_0p5',
         'train21bottom_left_val_minus_0p5','train14bottom_left_val_minus_0p5','train19Abottom_left_val_minus_0p5','train19Bbottom_left_val_minus_0p5','train24bottom_left_val_minus_0p5']

FIRST_TIME=True
for i in range(len(files)):
    conv_val_feat = load_array('/cat4/home/ubuntu/cat4/out/features/train/'+files[i]+'_sqznt.dat')
    labels_val = load_array('/cat/home/ubuntu/cat/in/train/labels/'+label_files[i]+'_labels.dat')
    if FIRST_TIME:
        predictions_all = lrg_model.predict(conv_val_feat) 
        predictions_all = np.hstack((predictions_all[0][:,0:15],predictions_all[1][:,0:4],predictions_all[2][:,0:2]))
        labels_all = labels_val
        FIRST_TIME = False
    else:
        temp = lrg_model.predict(conv_val_feat)
        temp = np.hstack((temp[0][:,0:15],temp[1][:,0:4],temp[2][:,0:2]))
        predictions_all = np.vstack((predictions_all,temp))
        labels_all = np.vstack((labels_all,labels_val))
    del(conv_val_feat,labels_val)
    gc.collect()
print predictions_all.shape, labels_all.shape
predictions_bottom_left = np.copy(predictions_all)

(31687, 21) (31687, 23)


In [12]:
labels_all = labels_all[:,2:23]
labels_all[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]] = labels_all[:,[0,5,6,9,10,11,12,13,14,15,16,17,18,19,20,1,2,3,4,7,8]]

In [13]:
predictionals_all_patches = np.zeros([5, 21,predictions_center.shape[0]],dtype=np.float32)
for i in range(21):
    predictionals_all_patches[0,i] = predictions_center[:,i]
    predictionals_all_patches[1,i] = predictions_top_left[:,i]
    predictionals_all_patches[2,i] = predictions_top_right[:,i]
    predictionals_all_patches[3,i] = predictions_bottom_right[:,i]
    predictionals_all_patches[4,i] = predictions_bottom_left[:,i]
print predictionals_all_patches.shape

(5, 21, 31687)


In [14]:
predictionals_all_patches_average = np.zeros([1, 21,predictions_center.shape[0]],dtype=np.float32)
for i in range(21):
    predictionals_all_patches_average[0,i] = (predictionals_all_patches[0,i] + predictionals_all_patches[1,i] + predictionals_all_patches[2,i] + predictionals_all_patches[3,i] + predictionals_all_patches[4,i])/5

In [15]:
predictionals_all_patches_average = predictionals_all_patches_average[0]
predictionals_all_patches_average = np.transpose(predictionals_all_patches_average,(1,0))

In [16]:
# Computing the area under the ROC curve
from sklearn.metrics import roc_curve, auc
from math import isnan

totalScore=0
count = 0

predictions_temp = np.copy(predictionals_all_patches_average)
truth_temp = np.copy(labels_all)

for j in range(21):   
    index = 0
    #remove rows with label = 0.5
    for i in range(labels_all.shape[0]):
        if labels_all[i,j] != 0.5:
            truth_temp[index,j]=labels_all[i,j]
            predictions_temp[index,j]=predictionals_all_patches_average[i,j]
            index += 1
    fpr, tpr, _ = roc_curve(truth_temp[0:index,j], predictions_temp[0:index,j])
    score = auc(fpr, tpr)
    print j, score
    if isnan(score):
        score=0.0
    else:
        totalScore+=score
        count +=1
    
print totalScore/count

0 0.496764622329
1 0.498478123018
2 0.997314026187
3 0.496432258914
4 0.999932642923
5 0.981278148222
6 0.967036042556
7 0.996522992279
8 0.987424017717
9 0.989334631048
10 0.997448591013
11 0.961565693138
12 0.910348332306
13 0.497203968154
14 0.756524164351
15 0.499899369381
16 0.500692136097
17 0.951198285249
18 0.946398710624
19 0.979275359802
20 0.960762959981
0.82723024168


# Visualize

In [ ]:
rgb_hires = load_array('/cataract/home/ubuntu/cataract/03_output/train21_rgb_hires.dat')
rgb_hires.shape

In [ ]:
labels = load_array('/cataract/home/ubuntu/cataract/03_output/train21_labels.dat')
labels[500,...]

In [ ]:
conv_trn_feat = load_array('/cataract4/home/ubuntu/cataract4/03_output/train21_fcf_feat.dat')

In [ ]:
labels.shape,conv_trn_feat.shape,rgb_hires.shape

In [ ]:
#load weights
l = lrg_model.layers
conv_fn = K.function([l[0].input, K.learning_phase()], l[-4].output)

In [ ]:
frame=500
inp = np.expand_dims(conv_trn_feat[frame], 0)
conv = conv_fn([inp,0])[0,:,:,0]
#conv = conv_fn([inp,0])
conv.shape

In [ ]:
def get_cm2(inp, label):
    conv = conv_fn([inp,0])[0,:,:,label]
    return scipy.misc.imresize(conv, (360,640))

In [ ]:
def plotImages(frame):
    from matplotlib.colors import LinearSegmentedColormap
    colors = ['red', 'brown', 'yellow', 'green', 'blue']
    cmap = LinearSegmentedColormap.from_list('name', colors)
    norm = plt.Normalize(0, 254)
    inp = np.expand_dims(conv_trn_feat[frame], 0)
    index_array = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    f = plt.figure(figsize=(10, 40))
    r = len(index_array)

    A = np.transpose(rgb_hires,(0,2,3,1))[frame,]

    for k, i in enumerate(index_array):
        for j , M in enumerate([A,A,A]):
            sp = f.add_subplot(r, 3, 3*k + j + 1)
            sp.axis('Off')
            if j==0:
                plt.imshow(M)
            else:
                if j==1:
                    plt.imshow(get_cm2(inp, i), cmap='cool', norm=norm, interpolation='none') 
                else:
                    plt.imshow(M)
                    plt.imshow(get_cm2(inp, i), cmap='cool', norm=norm, interpolation='none', alpha=0.5) 

256 convolutions filters

In [ ]:
plotImages(500)

In [ ]:
plotImages(1000)

In [ ]:
#plotImages(2000)

In [ ]:
#plotImages(3500)

In [ ]:
#plotImages(1500)

In [ ]:
labels[600,2:23]

In [ ]:
#plotImages(600)

128 convolution filters

In [ ]:
plotImages(100)

In [ ]:
plt.imshow(np.transpose(rgb_hires,(0,2,3,1))[1000,])

In [ ]:
cm = get_cm2(inp, 0)

In [ ]:
plt.imshow(cm, cmap="hot") # binary cross entropy with softmax

In [ ]:
gc.collect()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.transpose(rgb_hires,(0,2,3,1))[1000,])
plt.imshow(cm, cmap="hot", alpha=0.5) 

In [ ]:
cm = get_cm2(inp, 7)
plt.imshow(cm, cmap="gray") # binary cross entropy with softmax

In [ ]:
lr = [0.01,0.001,0.0001,0.00001]
e =[1,2,2,2]
p = [0.6]
tuneMyNw('fold_4','03_output/features',lr,e,p,get_lrg_layers,batch_size,allTags=True,base_model=None,early_stop=1)

In [ ]:
features, labels = loadFeatures('03_output/features','fold_4')
predictFolds('fold_4',get_bn_layers,'vgg',features,allTags=True,base_model='vgg')
np.savetxt('03_output/results/labels_vgg_fold_4.csv', labels, delimiter=",")

In [ ]:

tags = ['all']

learningSchedules = {}
learningSchedules['1'] = [0.01,0.001,0.0001]
learningSchedules['2'] = [0.01,0.001]
learningSchedules['3'] = [0.01]

epochSchedules = {}
epochSchedules['1'] = [1,2,4]
epochSchedules['2'] = [1,2]
epochSchedules['3'] = [1,2,3]
epochSchedules['4'] = [1,2,5]
epochSchedules['5'] = [1]
epochSchedules['6'] = [1,2,1]

dropouts = [0.8]
learningSchedule = ['1']
epochSchedule = ['1']

features, labels = loadFeatures('03_output/features','train')
trainFolds('test',tags,dropouts,learningSchedule,learningSchedules,epochSchedule,epochSchedules,
           get_bn_layers,'vgg',features, labels,batch_size,allTags=True,base_model='vgg')


In [ ]:
features, labels = loadFeatures('03_output/testfeatures', 'test')
predictFolds('test',get_bn_layers,'vgg',features,allTags=True,base_model='vgg')
np.savetxt('03_output/results/labels_vgg_test.csv', labels, delimiter=",")

# Scrapbook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io,transform
%matplotlib inline

def data_aug(img = img):
	mu = 0
	sigma = 0.1
	feature_vec=np.matrix(evecs_mat)

	# 3 x 1 scaled eigenvalue matrix
	se = np.zeros((3,1))
	se[0][0] = np.random.normal(mu, sigma)*evals[0]
	se[1][0] = np.random.normal(mu, sigma)*evals[1]
	se[2][0] = np.random.normal(mu, sigma)*evals[2]
	se = np.matrix(se)
	val = feature_vec*se

	# Parse through every pixel value.
	for i in xrange(img.shape[0]):
		for j in xrange(img.shape[1]):
			# Parse through every dimension.
			for k in xrange(img.shape[2]):
				img[i,j,k] = float(img[i,j,k]) + float(val[k])

imnames = ['n00.jpg','n01.jpg','n02.jpg','n03.jpg','n04.jpg','n05.jpg']
#load list of images
imlist = (io.imread_collection(imnames))

res = np.zeros(shape=(1,3))
for i in range(len(imlist)):
	# re-size all images to 256 x 256 x 3
	m=transform.resize(imlist[i],(256,256,3))
	# re-shape to make list of RGB vectors.
	arr=m.reshape((256*256),3)
	# consolidate RGB vectors of all images
	res = np.concatenate((res,arr),axis=0)
res = np.delete(res, (0), axis=0)

# subtracting the mean from each dimension
m = res.mean(axis = 0)
res = res - m

R = np.cov(res, rowvar=False)
print R

from numpy import linalg as LA
evals, evecs = LA.eigh(R)

idx = np.argsort(evals)[::-1]
evecs = evecs[:,idx]
# sort eigenvectors according to same index

evals = evals[idx]
# select the first 3 eigenvectors (3 is desired dimension
# of rescaled data array)

evecs = evecs[:, :3]
# carry out the transformation on the data using eigenvectors
# and return the re-scaled data, eigenvalues, and eigenvectors
m = np.dot(evecs.T, res.T).T

# perturbing color in image[0]
# re-scaling from 0-1
img = imlist[0]/255.0
data_aug(img)
plt.imshow(img)

In [ ]:
labelFile = load_array('/cataract/home/ubuntu/cataract/03_output/train01_labels.dat')
features = load_array('/cataract2/home/ubuntu/cataract2/03_output/train01_fcf_feat.dat')

In [ ]:
lrg_model.fit(conv_trn_feat, labels[:,2:23], batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, labels_val[:,2:23])) #binary cross entropy with sigmoid n = 256

In [ ]:
def auc_tool(truth_directory, prediction_directory, tool):
    """Computes the area under the ROC curve for one tool.
    """
    filename = ''
    try:
    truth = []
    predictions = []
    # loop on (truth, predictions) file pairs
    for file in range(1, num_files + 1):

    # getting the filenames
    if (file < 10):
    filename = file_prefix + '0{}.csv'.format(file)
    else:
    filename = file_prefix + '{}.csv'.format(file)
    truth_filename = join(truth_directory, filename)
    prediction_filename = join(prediction_directory, filename)
    # parsing the right column for the current tool
    truth_data = read_csv(truth_filename, header = 0, skipinitialspace = True,
    usecols = [tool], squeeze = True, dtype = 'float32').tolist()
    prediction_data = read_csv(prediction_filename, header = None, skipinitialspace = True,
    usecols = [tool], squeeze = True, dtype = 'float32').tolist()
    if len(truth_data) != len(prediction_data):
    raise ValueError('Files {} and {} have different row counts'.
    format(truth_filename, prediction_filename))

    # appending rows with consensual ground truth
    indices = [index for index, value in enumerate(truth_data) if value != 0.5]
    truth += [truth_data[index] for index in indices]
    predictions += [prediction_data[index] for index in indices]

    # computing the area under the ROC curve
    fpr, tpr, _ = roc_curve(truth, predictions)
    score = auc(fpr, tpr)
    return 0. if isnan(score) else score
    except Exception as e:
    print('Error: missing column in {} for tool number {}!'.format(filename, tool)
    if 'Usecols' in str(e) else 'Error: {}!'.format(e))
    return 0.


    def main():
    """Main function.
    """

    # parsing the command line
    parser = ArgumentParser(description = 'Evaluator for the CATARACTS challenge.')
    parser.add_argument('-t', '--truth', required = True, help = 'directory containing ground truth files')
    parser.add_argument('-p', '--predictions', required = True, help = 'directory containing automatic predictions')
    args = parser.parse_args()

    # computing tool-specific scores
    scores = []
    for tool in range(1, num_tools + 1):
    score = auc_tool(args.truth, args.predictions, tool)
    print('Score tool {0}: {1:.4f}'.format(tool, score))
    scores.append(score)

    # computing the average score
    print('Average: {0:.4f}'.format(sum(scores) / float(len(scores))))


In [ ]:
path='../03_output/'
name='train01'
rgb_hires = load_array(path + name + '_rgb_hires.dat')

In [ ]:
rgb_hires.shape

In [ ]:
conv_feat = vgg640.predict(rgb_hires[1:100], batch_size=32, verbose=1)

In [ ]:
rgb_hires = load_array('03_output/train01_rgb_hires.dat')
labels = load_array('03_output/train01_labels.dat')

In [ ]:
rgb_hires_val = load_array('03_output/train25_rgb_hires.dat')
labels_val = load_array('03_output/train25_labels.dat')

In [ ]:
rgb_hires_val.shape

In [ ]:
rgb_hires = np.reshape(np.transpose(rgb_hires,(1,0)),(2460,360,640,3)) 

In [ ]:
rgb_hires_val = np.reshape(np.transpose(rgb_hires_val,(1,0)),(6990,360,640,3)) 

In [ ]:
rgb_hires.shape, labels.shape,rgb_hires_val.shape, labels_val.shape

In [ ]:
rgb_hires = np.transpose(rgb_hires,(0,3,1,2)) 
rgb_hires_val = np.transpose(rgb_hires_val,(0,3,1,2))

In [ ]:
labels = np.transpose(labels,(1,0))
labels_val = np.transpose(labels_val,(1,0))

In [ ]:
labels.shape

In [ ]:
rgb_hires.shape
plt.imshow(rgb_hires[0])

In [ ]:
conv_trn_feat = vgg640.predict(rgb_hires_val, batch_size=32, verbose=1)
conv_val_feat = vgg640.predict(rgb_hires, batch_size=32, verbose=1)

In [ ]:
save_array('03_output/train25_fcn_feat.dat',conv_trn_feat)
save_array('03_output/train01_fcn_feat.dat',conv_val_feat)

In [ ]:
conv_layers,_ = split_at(vgg640, Convolution2D)

In [ ]:
nf=128; p=0.0

In [ ]:
nf=256; p=0.2

In [ ]:
nf=256; p=0.2
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        Convolution2D(21,3,3, border_mode='same'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

In [ ]:
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        Convolution2D(21,3,3, border_mode='same'),
        GlobalAveragePooling2D(),
        Activation('sigmoid')
    ]

In [ ]:
lrg_model = Sequential(get_lrg_layers())

In [ ]:
lrg_model.summary()

In [ ]:
lrg_model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 64

In [ ]:
lrg_model.fit(conv_trn_feat, labels_val[:,1:22], batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, labels[:,1:22])) #binary cross entropy with sigmoid n = 256

In [ ]:
lrg_model.fit(conv_trn_feat, labels_val[:,1:22], batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, labels[:,1:22])) #binary cross entropy with softmax n = 128

In [ ]:
lrg_model.fit(conv_trn_feat, labels_val[:,1:22], batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, labels[:,1:22])) #binary cross entropy with sigmoid n = 128

In [ ]:
lrg_model.optimizer.lr=1e-5

In [ ]:
lrg_model.fit(conv_trn_feat, labels_val[:,1:22], batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, labels[:,1:22])) #categorical cross entropy with softmax n = 128

In [ ]:
lrg_model.fit(conv_trn_feat, labels_val[:,1:22], batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, labels[:,1:22])) #binary cross entropy with softmax n = 128

In [ ]:
lrg_model.fit(conv_trn_feat, labels_val[:,1:22], batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, labels[:,1:22])) #binary cross entropy with sigmoid  n = 128

In [ ]:
lrg_model.fit(conv_trn_feat, labels_val[:,1:22], batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, labels[:,1:22])) #binary cross entropy with softmax n = 256

In [ ]:
l = lrg_model.layers
conv_fn = K.function([l[0].input, K.learning_phase()], l[-3].output)

In [ ]:
def get_cm2(inp, label):
    conv = conv_fn([inp,0])[0, label]
    return scipy.misc.imresize(conv, (360,640))

In [ ]:
inp = np.expand_dims(conv_val_feat[400], 0)

In [ ]:
plt.imshow(to_plot(rgb_hires[400]))

In [ ]:
cm = get_cm2(inp, 0)

In [ ]:
plt.imshow(cm, cmap="cool") # binary cross entropy with softmax

In [ ]:
plt.figure(figsize=(10,10))
plot(rgb_hires[400])
plt.imshow(cm, cmap="cool", alpha=0.5) #plt.imshow(cm, cmap="cool") # binary cross entropy with softmax

In [ ]:
plt.imshow(cm, cmap="cool") # binary cross entropy with softmax

In [ ]:
plt.figure(figsize=(10,10))
plot(rgb_hires[400])
plt.imshow(cm, cmap="cool", alpha=0.5) #plt.imshow(cm, cmap="cool") # binary cross entropy with softmax

In [ ]:
cm = get_cm2(inp, 2)

In [ ]:
plt.imshow(cm, cmap="gray") # binary cross entropy with softmax n = 256

In [ ]:
plt.figure(figsize=(10,10))
plot(rgb_hires[400])
plt.imshow(cm, cmap="gray", alpha=0.5) #plt.imshow(cm, cmap="cool") # binary cross entropy with softmax

In [ ]:
labels[400]

In [ ]:
files = ['train02','train03','train04','train05','train06','train07','train08','train09','train10','train11',
         'train12','train13','train14','train15','train16','train17','train18','train19','train20','train21','train22',
         'train23','train24','train25'] 

labels = load_array('../03_output/train01_labels.dat')
for i in tqdm(range(len(files))):
    lbl = load_array('../03_output/'+files[i]+'_labels.dat')
    labels = np.vstack((labels,lbl))
    del(lbl)
    gc.collect()
labels.shape

In [ ]:
anyLabel = sum(np.transpose(labels)[3:23,:])
unique, counts = np.unique(anyLabel, return_counts=True)
unique, counts

In [ ]:
labels[labels==0.5]=999
anyLabel = sum(np.transpose(labels)[3:23,:])
unique, counts = np.unique(anyLabel, return_counts=True)
unique, counts

In [ ]:
'0',36060/82479.0,'1',13838/82479.0,'2',30591/82479.0,'0.5 or 1.5 or 2.5 or 3', (1595+334+27+34)/82479.0

In [ ]:
anyLabel.shape

In [ ]:
from scipy.ndimage.interpolation import shift

lagggedLabels = shift(anyLabel, -15, cval=1)
lagggedLabels[lagggedLabels<0.5]=0
for i in range(30):
    temp = shift(anyLabel, -14+i, cval=1)
    temp[temp<0.5]=0
    lagggedLabels = np.vstack((lagggedLabels,temp))

lagggedLabels.shape
anyLabelLag15 = sum(lagggedLabels)
anyLabelLag15.shape
anyLabelLag15[anyLabelLag15>0]=1
unique, counts = np.unique(anyLabelLag15, return_counts=True)
unique, counts
keepOrNot = np.vstack((anyLabel,anyLabelLag15)).T
keepOrNot.shape

In [ ]:
unique, counts = np.unique(keepOrNot[:,0], return_counts=True)
unique, counts